In [ ]:
import pandas as pd

df = pd.read_csv('../september/result/september.csv')
df.head(20)

In [69]:
import numpy as np
import pandas as pd

df = pd.read_csv('../september/result/september.csv')


array = np.zeros((len(df), 10), dtype=int)

for i in range(len(df)):
    text_value = df.loc[i, 'text']
    if isinstance(text_value, str):  
        text_values = list(map(int, text_value.split()))
        length = min(len(text_values), 10)  
        array[i, :length] = text_values[:length]

# with open('output_array.txt', 'w') as f:
#     for row in array:
#         f.write(' '.join(map(str, row)) + '\n')  



In [70]:
import pandas as pd

df = pd.read_csv('../september/result/september.csv')

vector_accuracy  = [0] * len(df)

for i in range(len(df)):
    text_values = df.loc[i, 'emotion_scores'].strip('[]').split()  

    if text_values:
        first_emotion_score = float(text_values[6])  
        if first_emotion_score < 0.3:
            vector_accuracy [i] = 0
        else:
            vector_accuracy [i] = 1

# with open('vector_accuracy.txt', 'w') as f:
#     for value in vector_accuracy:
#         f.write(str(value) + '\n')  


In [71]:
labels = []
vector_w = [1000] * 10


for nums in array:
    temp = np.dot(nums, vector_w)
    if temp >= 0:
        labels.append(1)
    else:
        labels.append(0)

# with open('label.txt', 'w') as f:
#     for value in labels:
#         f.write(str(value) + '\n')  

In [72]:
vector_errors = []

for i in range(len(array)):
    vector_errors.append(vector_accuracy[i] - labels[i])

print("vector_errors: ", vector_errors)

# with open('vector_errors.txt', 'w') as f:
#     for value in vector_errors:
#         f.write(str(value) + '\n')  


vector_errors:  [0, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1, 0, -1, -1, 0, -1, 0, 0, -1, -1, -1, -1, 0, 0, -1, 0, -1, -1, -1, -1, 0, 0, 0, -1, -1, -1, 0, 0, -1, -1, -1, -1, -1, -1, -1, 0, -1, 0, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, -1, -1, 0, 0, -1, 0, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, -1, -1, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1, 0, -1, 0, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, 0, 0, 0, -1, -1, 0, -1, -1, 0, -1, -1, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,

In [75]:
results = []
for epoch in range(100):

    delta = [0] * 10

    for i in range(len(array)):
        for j in range(len(array[0])):
            delta[j] += vector_errors[i] * 0.1 * array[i][j]

    for i in range(10):
        vector_w[i] = vector_w[i] + delta[i]

    labels = []

    for nums in array:
        temp = np.dot(nums, vector_w)
        if temp >= 0:
            labels.append(1)
        else:
            labels.append(0)

    vector_errors = []

  
    for i in range(len(array)):
        vector_errors.append(vector_accuracy[i] - labels[i])

    # with open('vector_errors.txt', 'w') as f:
    #     for value in vector_errors:
    #         f.write(str(value) + '\n')  

    with open('epoh1.txt', 'a') as f:
        TP = sum((labels[i] == 1 and vector_accuracy[i] == 1) for i in range(len(labels)))
        TN = sum((labels[i] == 0 and vector_accuracy[i] == 0) for i in range(len(labels)))
        FP = sum((labels[i] == 1 and vector_accuracy[i] == 0) for i in range(len(labels)))
        FN = sum((labels[i] == 0 and vector_accuracy[i] == 1) for i in range(len(labels)))

        # Расчет метрик
        N = len(array)  # Общее количество наблюдений
        accuracy = (TP + TN) / N 
        precision = TP / (TP + FP) 
        recall = TP / (TP + FN) 
        Fscore = TP / (TP + FN + FP) 

        # Запись результатов в файл
        f.write(f'Эпоха {epoch} ------------------------------------\n')
        f.write(f'accuracy = {accuracy:.4f}\n')
        f.write(f'precision = {precision:.4f}\n')
        f.write(f'recall = {recall:.4f}\n')
        f.write(f'Fscore = {Fscore:.4f}\n\n')

    results.append({
        'Epoch': epoch,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'Fscore': Fscore
    })

results_df = pd.DataFrame(results)
results_df.to_csv('./ignore/results_analysis.csv', index=False)


In [76]:
import numpy as np
import pandas as pd

df = pd.read_csv('../september/result/september.csv')


array = np.zeros((len(df), 10), dtype=int)

for i in range(len(df)):
    text_value = df.loc[i, 'text']
    if isinstance(text_value, str):  
        text_values = list(map(int, text_value.split()))
        length = min(len(text_values), 10)  
        array[i, :length] = text_values[:length]

# with open('output_array.txt', 'w') as f:
#     for row in array:
#         f.write(' '.join(map(str, row)) + '\n')  



In [77]:
import pandas as pd

df = pd.read_csv('../september/result/september.csv')

vector_accuracy  = [0] * len(df)

for i in range(len(df)):
    text_values = df.loc[i, 'emotion_scores'].strip('[]').split()  

    if text_values:
        first_emotion_score = float(text_values[6])  
        if first_emotion_score < 0.3:
            vector_accuracy [i] = 0
        else:
            vector_accuracy [i] = 1

# with open('vector_accuracy.txt', 'w') as f:
#     for value in vector_accuracy:
#         f.write(str(value) + '\n')  


In [78]:
import numpy as np

labels = []
vector_w = [1000] * 10


for nums in array:
    temp = np.dot(nums, vector_w)
    labels.append(round(1 / (1 + np.exp(-temp))))

# with open('label.txt', 'w') as f:
#     for value in labels:
#         f.write(str(value) + '\n')  

In [79]:
vector_errors = []

for i in range(len(array)):
    vector_errors.append(vector_accuracy[i] - labels[i])

print("vector_errors: ", vector_errors)

# with open('vector_errors.txt', 'w') as f:
#     for value in vector_errors:
#         f.write(str(value) + '\n')  


vector_errors:  [0, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1, 0, -1, -1, 0, -1, 0, 0, -1, -1, -1, -1, 0, 0, -1, 0, -1, -1, -1, -1, 0, 0, 0, -1, -1, -1, 0, 0, -1, -1, -1, -1, -1, -1, -1, 0, -1, 0, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, -1, -1, 0, 0, -1, 0, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, -1, -1, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1, 0, -1, 0, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, 0, 0, 0, -1, -1, 0, -1, -1, 0, -1, -1, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,

In [81]:
results = []
for epoch in range(100):

    delta = [0] * 10
    delta_w = 0

    for i in range(len(array)):
        for j in range(len(array[0])):
            delta[j] += vector_errors[i] * 0.1 * array[i][j]
            delta_w += vector_errors[i] * 0.1 

    for i in range(10):
        vector_w[i] = vector_w[i] + delta[i]

    labels = []

    for nums in array:
        temp = np.dot(nums, vector_w)
        labels.append(round(1 / (1 + np.exp(-temp))))

    vector_errors = []
  
    for i in range(len(array)):
        vector_errors.append(vector_accuracy[i] - labels[i])

    # with open('vector_errors.txt', 'w') as f:
    #     for value in vector_errors:
    #         f.write(str(value) + '\n')  

    with open('epoh2.txt', 'a') as f:
        TP = sum((labels[i] == 1 and vector_accuracy[i] == 1) for i in range(len(labels)))
        TN = sum((labels[i] == 0 and vector_accuracy[i] == 0) for i in range(len(labels)))
        FP = sum((labels[i] == 1 and vector_accuracy[i] == 0) for i in range(len(labels)))
        FN = sum((labels[i] == 0 and vector_accuracy[i] == 1) for i in range(len(labels)))

        # Расчет метрик
        N = len(array)  # Общее количество наблюдений
        accuracy = (TP + TN) / N 
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        Fscore = TP / (TP + FN + FP) if (TP + FN + FP) > 0 else 0

        # Запись результатов в файл
        f.write(f'Эпоха {epoch} ------------------------------------\n')
        f.write(f'accuracy = {accuracy:.4f}\n')
        f.write(f'precision = {precision:.4f}\n')
        f.write(f'recall = {recall:.4f}\n')
        f.write(f'Fscore = {Fscore:.4f}\n\n')

    results.append({
        'Epoch': epoch,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'Fscore': Fscore
    })

results_df = pd.DataFrame(results)
results_df.to_csv('./ignore/results_analysis_sigmoida.csv', index=False)


/var/folders/4z/x3rxm6xs5c5_01z63xh1mdhm0000gn/T/ipykernel_11655/1435012619.py:19: RuntimeWarning: overflow encountered in exp
  labels.append(round(1 / (1 + np.exp(-temp))))


In [86]:
import pandas as pd

# Загрузка данных из CSV файлов
file1 = './ignore/results_analysis.csv'
file2 = './ignore/results_analysis_sigmoida.csv'

data1 = pd.read_csv(file1)
data2 = pd.read_csv(file2)

# Вычисление средних значений метрик
metrics1 = data1[['Accuracy', 'Precision', 'Recall', 'Fscore']].mean()
metrics2 = data2[['Accuracy', 'Precision', 'Recall', 'Fscore']].mean()

# Вычисление стандартного отклонения метрик
std_metrics1 = data1[['Accuracy', 'Precision', 'Recall', 'Fscore']].std()
std_metrics2 = data2[['Accuracy', 'Precision', 'Recall', 'Fscore']].std()

# Вывод результатов
print("Средние значения метрик для первого файла:")
print(metrics1)
print("Стандартные отклонения метрик для первого файла:")
print(std_metrics1)

print("\nСредние значения метрик для второго файла:")
print(metrics2)
print("Стандартные отклонения метрик для второго файла:")
print(std_metrics2)

# Сравнение метрик
for metric in metrics1.index:
    if metrics1[metric] > metrics2[metric]:
        print(f"{metric} больше при линейной классификации.")
    elif metrics1[metric] < metrics2[metric]:
        print(f"{metric} больше при сигмоидальной классификации.")
    else:
        print(f"{metric} одинаков при обоих классификациях.")


Средние значения метрик для первого файла:
Accuracy     0.629749
Precision    0.521952
Recall       0.274243
Fscore       0.097548
dtype: float64
Стандартные отклонения метрик для первого файла:
Accuracy     0.187325
Precision    0.154655
Recall       0.399680
Fscore       0.113528
dtype: float64

Средние значения метрик для второго файла:
Accuracy     0.627765
Precision    0.190075
Recall       0.273135
Fscore       0.094817
dtype: float64
Стандартные отклонения метрик для второго файла:
Accuracy     0.190828
Precision    0.220232
Recall       0.404614
Fscore       0.113449
dtype: float64
Accuracy больше при линейной классификации.
Precision больше при линейной классификации.
Recall больше при линейной классификации.
Fscore больше при линейной классификации.
